In [15]:
!pip uninstall politician2vec -y

Found existing installation: politician2vec 0.0.1
Uninstalling politician2vec-0.0.1:
  Successfully uninstalled politician2vec-0.0.1


In [16]:
!pip install --upgrade git+ssh://git@github.com/mathiasbruun/politician2vec.git

  Cloning ssh://****@github.com/mathiasbruun/politician2vec.git to /private/var/folders/8q/02vc3fzn3r1fv7wzycyspjz80000gn/T/pip-req-build-85eba12g
  Running command git clone -q 'ssh://****@github.com/mathiasbruun/politician2vec.git' /private/var/folders/8q/02vc3fzn3r1fv7wzycyspjz80000gn/T/pip-req-build-85eba12g
  Created wheel for politician2vec: filename=politician2vec-0.0.1-py3-none-any.whl size=26678 sha256=12f84fc96cc404218f84044cb0936d46bd655f1cfa796d1e2d3949912a626f8e
  Stored in directory: /private/var/folders/8q/02vc3fzn3r1fv7wzycyspjz80000gn/T/pip-ephem-wheel-cache-70hck797/wheels/ac/44/62/b0b3ddf2882cd1b1d1cc4e060c5c525b951ae01496d65cd472
Successfully built politician2vec


In [53]:
from politician2vec.Politician2Vec import Politician2Vec
from politician2vec.utils import *
import pickle
import pandas as pd
import numpy as np
import multiprocessing
available_workers = multiprocessing.cpu_count()

In [54]:
test_data_path = 'data/clean/combined/subset_party_imputed.p'

In [55]:
with open(test_data_path, 'rb') as p:
    test_data = pickle.load(p)

In [56]:
test_data.shape

(349594, 7)

In [57]:
tw_data = test_data.loc[
    test_data['source'] == 'twitter'
]

In [58]:
tw_data['party'].value_counts(dropna = False)

S      54566
V      37099
EL     27629
RV     22041
SF     21593
ALT    18070
LA     14566
KF     13241
DF     12363
FG      4196
NB      1834
M        157
K          2
Name: party, dtype: int64

In [62]:
grouped_docs = tw_data.groupby(['full_name', 'party'])['doc'].apply('. '.join).reset_index()

In [66]:
#docs = [doc for doc in tw_data.doc]
docs = [doc for doc in grouped_docs.doc]
parties = np.array([party for party in grouped_docs.party])

In [67]:
pol2vec_model = Politician2Vec(
    documents = docs,
    custom_clusters = parties,
    tokenizer = preproc_docs,
    embedding_model = 'doc2vec',
    min_count = 50,
    speed = 'fast-learn',
    workers = available_workers
    #hdbscan_min_cluster_size = 15,
    #workers=4,
    #doc2vec_vector_size = 300,
    #doc2vec_window = 8,
    #doc2vec_samples_threshold = 1e-5
)

2022-11-12 09:32:45,132 - politician2vec - INFO - Pre-processing documents for training
2022-11-12 09:33:17,969 - politician2vec - INFO - Creating joint document/word embedding
2022-11-12 09:38:13,886 - politician2vec - INFO - ONLY creating UMAP projection, NOT HDBSCAN!!!
2022-11-12 09:38:22,483 - politician2vec - INFO - Finding topics


In [68]:
def inspect_topic(top2vec, topic_idx, n_docs=None, query_substr=None):
    '''
    Print top words and top docs for a given
    topic.
    -------
    manual_num (int):  manually assigned topic number (i.e. 1-indexed).
    
    n_docs (int, optional): n top documents to print for a given topic.
        Default is to print all docs within a given topic.
    
    query_substr (str, optional): if specified, only documents containing
        this substring will be printed. Cannot be specified with n_docs,
        as this would return only results within a subset of topic docs.
    '''

    num_topics = top2vec.get_num_topics()
    topic_words, word_scores, topic_nums = top2vec.get_topics(num_topics)

    # Get topic sizes so we know max n docs
    topic_sizes, topic_nums = top2vec.get_topic_sizes()
    docs_to_return = topic_sizes[topic_idx]

    # Override n docs to return, if specified
    if n_docs:
        docs_to_return = n_docs

    # Get docs for input topic id
    documents, document_scores, document_ids = top2vec.search_documents_by_topic(
        topic_num=topic_idx,
        num_docs=docs_to_return
        )

    # Limit output to docs containign certain substring, if specified
    if query_substr and n_docs:
        raise Exception('Please do NOT specify n_docs with substring query!\nOtherwise the search is only carried out for a subset of topic docs.')
    
    # Throw exception if substring query attempted on subset of docs!
    elif query_substr:
        documents = [doc for doc in documents if query_substr in doc.lower()]

    # Print output
    print('--- TOP 50 WORDS ---\n', topic_words[topic_idx], '\n')

    print(f'--- TOP {docs_to_return} DOCS. SUBSTRING QUERY: {query_substr} (n = {len(documents)}) ---\n', documents)

In [69]:
word_vectors = pol2vec_model.model.wv

In [81]:
word_vectors.most_similar(positive = ['minister'])

[('to', 0.5383712649345398),
 ('ministre', 0.5155658721923828),
 ('danish', 0.5089468359947205),
 ('a', 0.49930647015571594),
 ('denmark', 0.4918190836906433),
 ('with', 0.48776963353157043),
 ('on', 0.48716092109680176),
 ('sagen', 0.48335105180740356),
 ('’', 0.4816931188106537),
 ('and', 0.477433443069458)]

In [191]:
inspect_topic(pol2vec_model, 0, n_docs=1, query_substr=None)

--- TOP 50 WORDS ---
 ['sophieloehde' 'jakobellemann' 'venstredk' 'martingeertsen'
 'torstenschack' 'mortendahlin' 'hansandersenv' 'venstre' 'janeheitmann'
 'ullegaard' 'janejoergensen' 'ellentranenorby' 'kristianjensen'
 'louiseelholm' 'troelslundp' 'venstres' 'marleneambo' 'thdanielsen'
 'mariebjerre' 'danskerne' 'stephanielose' 's-regeringen' 'bankheidi'
 'christoffermels' 'madsfuglede' 'skattestigninger' 'kimvalentindk'
 'mette' 'michaelaastrup' 'skatter' 'larsloekke' 'linealidell' '💙'
 'frederiksen' 'annimatthiesen' 'kplorentzen' 'larsclilleholt' 'vlm'
 'ydelserne' 'aahlers' 'stemlauritzen' 'erlingbonnesen' 'frederiksens'
 'skattestop' 'mlundbyhansen' 'dkaid' 'arbejdspladser' 's-regeringens'
 'tomjensen' 'regeringen'] 

--- TOP 1 DOCS. SUBSTRING QUERY: None (n = 1) ---
 ['RT @MortenDahlin: Et både ærligt og rammende interview af formand for @venstredk @JakobEllemann. Det kaster lys over en væsentlig del af mi…. @nielsthdahl Og jeg kender den søde politibetjent - han er rigtig sød 

In [85]:
model_path = 'embedding_models/politician2vec_test_v1.txt'

In [86]:
pol2vec_model.save(model_path)

# First `politician2vec` demo

## To do
- Descriptive statistics/viz for entire dataset
- Settle on centroid calculation method (handle outliers better, remove alt method?)
- Compile populism-related words for constructing axes
- Determine possibilities of comparative analysis
- Consistently change "topic" to "party"/"cluster" or the like

In [1]:
!pip uninstall politician2vec -y

Found existing installation: politician2vec 0.0.1
Uninstalling politician2vec-0.0.1:
  Successfully uninstalled politician2vec-0.0.1


In [2]:
!pip install --upgrade git+ssh://git@github.com/mathiasbruun/politician2vec.git

  Cloning ssh://****@github.com/mathiasbruun/politician2vec.git to /private/var/folders/8q/02vc3fzn3r1fv7wzycyspjz80000gn/T/pip-req-build-u70tct1u
  Running command git clone -q 'ssh://****@github.com/mathiasbruun/politician2vec.git' /private/var/folders/8q/02vc3fzn3r1fv7wzycyspjz80000gn/T/pip-req-build-u70tct1u
  Created wheel for politician2vec: filename=politician2vec-0.0.1-py3-none-any.whl size=26812 sha256=e4438ad92b7987d7f2fb413f19c7859c994e02fea772856cc9ace4fa22aa2f20
  Stored in directory: /private/var/folders/8q/02vc3fzn3r1fv7wzycyspjz80000gn/T/pip-ephem-wheel-cache-kj72_zz0/wheels/ac/44/62/b0b3ddf2882cd1b1d1cc4e060c5c525b951ae01496d65cd472
Successfully built politician2vec


In [3]:
from politician2vec import Politician2Vec
from politician2vec.utils import *
import pickle
import pandas as pd
import numpy as np
import multiprocessing
available_workers = multiprocessing.cpu_count()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
test_data_path = 'data/clean/combined/subset_party_imputed.p'

In [5]:
with open(test_data_path, 'rb') as p:
    test_data = pickle.load(p)

test_data.shape

(349594, 7)

In [6]:
tw_data = test_data.loc[
    test_data['source'] == 'twitter'
]

In [7]:
tw_data['party'].value_counts(dropna = False)

S      54566
V      37099
EL     27629
RV     22041
SF     21593
ALT    18070
LA     14566
KF     13241
DF     12363
FG      4196
NB      1834
M        157
K          2
Name: party, dtype: int64

In [27]:
grouped_docs = tw_data.groupby(['full_name', 'party'])['doc'].apply('. '.join).reset_index()

In [10]:
#docs = [doc for doc in tw_data.doc]
docs = [doc for doc in grouped_docs.doc]
parties = np.array([party for party in grouped_docs.party])

In [30]:
pol2vec_model = Politician2Vec(
    documents = docs,
    custom_clusters = parties,
    party_inference_method = 'mean',
    tokenizer = preproc_docs,
    embedding_model = 'doc2vec',
    min_count = 50,
    speed = 'deep-learn',
    workers = available_workers
    #doc2vec_vector_size = 300,
    #doc2vec_window = 8,
    #doc2vec_samples_threshold = 1e-5
)

2022-11-13 11:15:14,980 - politician2vec - INFO - Pre-processing documents for training
2022-11-13 11:15:48,210 - politician2vec - INFO - Creating joint document/word embedding
2022-11-13 12:22:54,887 - politician2vec - INFO - Projecting vectors to 5D space using UMAP (HDBSCAN clustering disabled!)
2022-11-13 12:23:04,438 - politician2vec - INFO - Estimating party positions using mean...
2022-11-13 12:23:04,473 - politician2vec - INFO - All done!


In [35]:
# TODO: This should probably be implemented as a method of the Politician2Vec class
def inspect_topic(politician2vec_model, topic_idx, n_docs=None, query_substr=None):
    '''
    Print top words and top docs for a given
    topic.
    -------
    manual_num (int):  manually assigned topic number (i.e. 1-indexed).
    
    n_docs (int, optional): n top documents to print for a given topic.
        Default is to print all docs within a given topic.
    
    query_substr (str, optional): if specified, only documents containing
        this substring will be printed. Cannot be specified with n_docs,
        as this would return only results within a subset of topic docs.
    '''

    num_topics = politician2vec_model.get_num_topics()
    topic_words, word_scores, topic_nums = politician2vec_model.get_topics(num_topics)

    # Get topic sizes so we know max n docs
    topic_sizes, topic_nums = politician2vec_model.get_topic_sizes()
    docs_to_return = topic_sizes[topic_idx]

    # Override n docs to return, if specified
    if n_docs:
        docs_to_return = n_docs

    # Get docs for input topic id
    documents, document_scores, document_ids = politician2vec_model.search_documents_by_topic(
        topic_num=topic_idx,
        num_docs=docs_to_return
        )

    # Limit output to docs containign certain substring, if specified
    if query_substr and n_docs:
        raise Exception('Please do NOT specify n_docs with substring query!\nOtherwise the search is only carried out for a subset of topic docs.')
    
    # Throw exception if substring query attempted on subset of docs!
    elif query_substr:
        documents = [doc for doc in documents if query_substr in doc.lower()]

    # Print output
    print('--- TOP 50 WORDS ---\n', topic_words[topic_idx], '\n')

    print(f'--- TOP {docs_to_return} DOCS. SUBSTRING QUERY: {query_substr} (n = {len(documents)}) ---\n', documents)

In [36]:
word_vectors = pol2vec_model.model.wv

In [37]:
word_vectors.most_similar(positive = ['mette'])

[('frederiksen', 0.9280436038970947),
 ('f', 0.7573208212852478),
 ('frederiksens', 0.7178287506103516),
 ('statsminister', 0.7096027731895447),
 ('statsministeren', 0.6731045842170715),
 ('regering', 0.6187393069267273),
 ('siger', 0.5733619332313538),
 ('svare', 0.5723803639411926),
 ('svar', 0.5638641119003296),
 ('statsministerens', 0.5554270148277283)]

In [154]:
inspect_topic(pol2vec_model, 10, n_docs=1, query_substr=None)

--- TOP 50 WORDS ---
 ['sikandasiddique' 'susannezimmer' 'nikogrunfeld' 'ebjerkrheim'
 'yadvindersingh' 'friegronne' 'uffeelbaek' 'brydfri' 'rajeshholmen'
 'frie' 'støttepartierne' 'klimakrisen' 'planeten' 'racisme'
 'klimahandling' 'minoriteter' 'sikandar' 'flygtningepolitik' 'apartheid'
 'animalske' 'muslimer' 'brydmedantiklimaaktivisterne' 'grønne'
 'dkgreenrt' 'larskohler' 'stedet' 'diskrimination' 'profilering'
 'portland' 'vælgererklæring' 'udleder' 'nødt' 'flygtninge' 'fg'
 'helenehagel' 'kaldt' '😇' 'lynetteholm' 'planet' 'ødelæggende' '🤷'
 'greenpeacedk' 'co' 'fuldstændigt' 'demokratisk' 'motorveje' 'etnicitet'
 'fossile' 'greenwashing' 'klimaforandringerne'] 

--- TOP 1 DOCS. SUBSTRING QUERY: None (n = 1) ---
 ['Den nuværende animalske produktion er skadelig for klimaet, og derfor foreslår vi i FG’s klimavision at reducere den animalske produktion med 95 procent i 2030. https://t.co/bHkmfKjqCV|60 kilo foder og 100-150 liter vand for 30 liter mælk virker som en dårlig måde at p

In [44]:
model_path = 'embedding_models/politician2vec_test_deeplearn.txt'

In [45]:
pol2vec_model.save(model_path)

### Viz dev below

In [108]:
pol2vec_model, doc2vec_model = load_politician2vec_from_txt(model_path)

Loading Politician2Vec model...
Retrieving document embedding...
All done!


In [109]:
doc2vec2tensor(
    doc2vec_model,
    temp_w2v_path = 'tensorboard_input/temp/doc_tensor_deeplearn.w2v',
    tsv_prefix = 'tensorboard_input/deeplearn',
    output_docvecs = True,
    output_wordvecs = False
)

You have elected to extract only document vectors.
Please note that further preprocessing -- such as filtering based on topics of
interest -- may be desired in order to facilitate TensorBoard visualisation.
Please see get_doc_topic_df(), vector_subset2tensor_without_words(), and
metadata2tensor()

Saving temp w2v file and converting to tensor. This may take a while...


2022-11-13 14:17:56,729 - word2vec2tensor - INFO - running /Users/mathiasbruun/me/anaconda3/lib/python3.7/site-packages/gensim/scripts/word2vec2tensor.py -i tensorboard_input/temp/doc_tensor_deeplearn.w2v -o tensorboard_input/deeplearn
2022-11-13 14:17:56,730 - keyedvectors - INFO - loading projection weights from tensorboard_input/temp/doc_tensor_deeplearn.w2v
2022-11-13 14:17:56,765 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (154, 300) matrix of type float32 from tensorboard_input/temp/doc_tensor_deeplearn.w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-11-13T14:17:56.752679', 'gensim': '4.1.2', 'python': '3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 05:57:50) \n[Clang 11.1.0 ]', 'platform': 'Darwin-21.4.0-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
2022-11-13 14:17:56,808 - word2vec2tensor - INFO - 2D tensor file saved to tensorboard_input/deeplearn_tensor.tsv
2022-11-13 14:17:56,808 - word2vec2tensor - INFO - Tensor metadata fil

In [110]:
n_words = len(doc2vec_model.wv)
n_docs = len(doc2vec_model.dv)
vocab = pol2vec_model.vocab

In [155]:
topic_labels = {
    0: 'Venstre',
    1: 'Socialdemokratiet',
    2: 'Dansk_Folkeparti',
    3: 'Enhedslisten',
    4: 'Radikale_Venstre',
    5: 'SF',
    6: 'Konservative',
    7: 'Liberal_Alliance',
    8: 'Alternativet',
    9: 'Nye_Borgerlige',
    10: 'Frie_Grønne',
    11: 'Kristendemokraterne',
    12: 'Moderaterne'
}

In [156]:
topic_df = get_doc_topic_df(
    pol2vec_model,
    no_substantive_topics = 13,
    snippets = True
)

Topic sizes before filtering (topic 13 is "Other"):

[[ 0 37]
 [ 1 32]
 [ 2 16]
 [ 3 14]
 [ 4 13]
 [ 5 11]
 [ 6 11]
 [ 7  6]
 [ 8  5]
 [ 9  5]
 [10  2]
 [11  1]
 [12  1]]


In [157]:
topic_df

,doc,top,snippet
0,0,7,Det offentlige forbrug er steget med 9 mia. kr...
1,1,8,RT @TFKortholm: Vi er klar til valgkampen 💚🖖 S...
2,2,8,Vores super seje @OlumekoC er på den internati...
3,3,1,RT @DSU_1920: Statsministeren underskriver DSU...
4,4,4,"Hvad er egentlig @SorenPape, @KonservativeDK, ..."
...,...,...,...
149,149,0,RT @JakobEllemann: Danskerne drømmer om at bo ...
150,150,0,RT @AndreasBachM: 75.135 ‼️ gange tak til alle...
151,151,0,En borgerlig statsminister forudsætter et borg...
152,152,1,"@JackPGB Ja, man får det dårligt 😔|Det her gør..."


In [158]:
# TODO: Okay, we have clearly made a mistake. By swapping party classes for politicians,
# we are now telling the model to find politician-level centroids wrt. politician superdocs, yielding...
# ... well, obviously, 154 centroids, correspodning to the 154 super documents. That makes no sense,
# since the centroid and the superdocument will be the same by construction.
#
# We either want document-level centroids clustered by politician OR politician-level centroids clustered
# by party. So politician superdocs + input party label array.
#
# TODO: Also extract topic vectors themselves, which we usually never do!
#
# TODO: Calculate medioid instead of centroid? To account for noisy estimates of politician positions?
#
# TODO: Perhaps chunk the politician-level superdocuments according to a certain special character used
# to join them? Could increase performance in doc2vec.

In [159]:
topic_df

,doc,top,snippet
0,0,7,Det offentlige forbrug er steget med 9 mia. kr...
1,1,8,RT @TFKortholm: Vi er klar til valgkampen 💚🖖 S...
2,2,8,Vores super seje @OlumekoC er på den internati...
3,3,1,RT @DSU_1920: Statsministeren underskriver DSU...
4,4,4,"Hvad er egentlig @SorenPape, @KonservativeDK, ..."
...,...,...,...
149,149,0,RT @JakobEllemann: Danskerne drømmer om at bo ...
150,150,0,RT @AndreasBachM: 75.135 ‼️ gange tak til alle...
151,151,0,En borgerlig statsminister forudsætter et borg...
152,152,1,"@JackPGB Ja, man får det dårligt 😔|Det her gør..."


In [160]:
topic_df['snippet'] = grouped_docs['full_name']

In [161]:
topic_df

,doc,top,snippet
0,0,7,Alex Vanopslagh
1,1,8,Alternativet
2,2,8,Alternativets Unge
3,3,1,Anders Kronborg
4,4,4,Andreas Steenberg
...,...,...,...
149,149,0,Ulla Tørnæs
150,150,0,Venstre
151,151,0,Venstres Ungdom
152,152,1,Yildiz Akdogan


In [162]:
len(pol2vec_model.topic_vectors)

13

In [163]:
metadata2tensor(
    topic_df,
    metadata_path = 'tensorboard_input/deeplearn_metadata.tsv',
    label_list = topic_labels
)

In [164]:
pol2vec_model.topic_vectors

array([[ 0.14969347,  0.07628008, -0.03789625, ...,  0.09185682,
        -0.03362478, -0.00900998],
       [-0.01559262,  0.05238951, -0.07623937, ..., -0.03731279,
        -0.06737081, -0.04292313],
       [ 0.12063742,  0.02928232,  0.07233113, ...,  0.00725271,
        -0.10043973, -0.06223   ],
       ...,
       [-0.03668575,  0.07050475, -0.13768224, ..., -0.04055063,
        -0.0225606 , -0.067119  ],
       [-0.07218929, -0.03267358,  0.11535928, ..., -0.04710085,
         0.0269724 , -0.02580858],
       [ 0.03947576,  0.02787272, -0.02921587, ..., -0.16664107,
        -0.10845865,  0.04946344]], dtype=float32)

In [165]:
# test read of tensor output
with open('tensorboard_input/deeplearn_tensor.tsv','r') as r:
    lines_test = r.readlines()

len(lines_test)

154

In [166]:
partyvecs2tensor(
    party_vecs = pol2vec_model.topic_vectors,
    out_path = 'tensorboard_input/deeplearn_parties.tsv'
)

In [167]:
def partyvecs2tensor(party_vecs, out_path):
    vec_strs = []

    for vec in party_vecs:
        vec_str = ''.join([str(val) +'\t' for val in vec]).rstrip('\t')
        vec_strs.append(vec_str)

    tensor_str = '\n'.join(vec_strs)
    
    with open(out_path, 'w') as f:
        f.write(tensor_str)

In [168]:
def concat_tensors(tensor_files, out_path):
    tensor_strs = []
    
    for file in tensor_files:
        tensor_str = pd.read_csv(file, sep = '\t', header = None)
        tensor_strs.append(tensor_str)
    
    concat_tensor = pd.concat(tensor_strs)
    
    concat_tensor.to_csv(out_path, sep = '\t', header = False, index = False)

In [169]:
label_df = pd.DataFrame(topic_labels, index = ['doc']).T
#label_df['topic'] = ''

In [170]:
label_df.to_csv(
    'tensorboard_input/deeplearn_parties_metadata.tsv',
    sep = '\t',
    header = False,
    index = False
)

In [171]:
# Combine docvecs and partyvecs
politician_file = 'tensorboard_input/deeplearn_tensor.tsv'
party_file = 'tensorboard_input/deeplearn_parties.tsv'

tensor_files = [politician_file, party_file]
concat_tensors(tensor_files, 'tensorboard_input/deeplearn_combined.tsv')

In [172]:
# Combine metadata
politician_file = 'tensorboard_input/deeplearn_metadata.tsv'
party_file = 'tensorboard_input/deeplearn_parties_metadata.tsv'

tensor_files = [politician_file, party_file]
concat_tensors(tensor_files, 'tensorboard_input/deeplearn_combined_metadata.tsv')